In [1]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
from collections import Counter #for Smote, 
import warnings

Using TensorFlow backend.


In [2]:
warnings.filterwarnings("ignore")
trainData = pd.read_csv('E:/Kaggle/homesite-quote-conversion/train.csv')
testData = pd.read_csv('E:/Kaggle/homesite-quote-conversion/test.csv')
print(trainData.shape)
print(testData.shape)

(260753, 299)
(173836, 298)


In [3]:
train=trainData.copy()
train.drop(['QuoteConversion_Flag'],axis=1,inplace=True)
train.head()

,QuoteNumber,Original_Quote_Date,Field6,Field7,Field8,Field9,Field10,Field11,Field12,CoverageField1A,...,GeographicField59A,GeographicField59B,GeographicField60A,GeographicField60B,GeographicField61A,GeographicField61B,GeographicField62A,GeographicField62B,GeographicField63,GeographicField64
0,1,8/16/2013,B,23,0.9403,0.0006,965,1.0200,N,17,...,9,9,-1,8,-1,18,-1,10,N,CA
1,2,4/22/2014,F,7,1.0006,0.0040,548,1.2433,N,6,...,10,10,-1,11,-1,17,-1,20,N,NJ
2,4,8/25/2014,F,7,1.0006,0.0040,548,1.2433,N,7,...,15,18,-1,21,-1,11,-1,8,N,NJ
3,6,4/15/2013,J,10,0.9769,0.0004,1165,1.2665,N,3,...,6,5,-1,10,-1,9,-1,21,N,TX
4,8,1/25/2014,E,23,0.9472,0.0006,1487,1.3045,N,8,...,18,22,-1,10,-1,11,-1,12,N,IL


In [4]:
train['Date'] = pd.to_datetime(pd.Series(train['Original_Quote_Date']))
testData['Date'] = pd.to_datetime(pd.Series(testData['Original_Quote_Date']))

In [6]:
#train['Date1']=train['Date'].apply(lambda x: str(pd.datetime.now().date()-x.date()))
train['Year']=train['Date'].apply(lambda x: int(str(x)[:4]))
train['Month']=train['Date'].apply(lambda x: int(str(x)[5:7]))
testData['Year']=testData['Date'].apply(lambda x: int(str(x)[:4]))
testData['Month']=testData['Date'].apply(lambda x: int(str(x)[5:7]))
train.head()

,QuoteNumber,Original_Quote_Date,Field6,Field7,Field8,Field9,Field10,Field11,Field12,CoverageField1A,...,GeographicField60B,GeographicField61A,GeographicField61B,GeographicField62A,GeographicField62B,GeographicField63,GeographicField64,Date,Year,Month
0,1,8/16/2013,B,23,0.9403,0.0006,965,1.0200,N,17,...,8,-1,18,-1,10,N,CA,2013-08-16,2013,8
1,2,4/22/2014,F,7,1.0006,0.0040,548,1.2433,N,6,...,11,-1,17,-1,20,N,NJ,2014-04-22,2014,4
2,4,8/25/2014,F,7,1.0006,0.0040,548,1.2433,N,7,...,21,-1,11,-1,8,N,NJ,2014-08-25,2014,8
3,6,4/15/2013,J,10,0.9769,0.0004,1165,1.2665,N,3,...,10,-1,9,-1,21,N,TX,2013-04-15,2013,4
4,8,1/25/2014,E,23,0.9472,0.0006,1487,1.3045,N,8,...,10,-1,11,-1,12,N,IL,2014-01-25,2014,1


In [7]:
train.drop(['Date','Original_Quote_Date'],axis=1,inplace=True)
testData.drop(['Date','Original_Quote_Date'],axis=1,inplace=True)

In [8]:
total=train.isnull().sum().sort_values(ascending = False)
percent = (train.isnull().sum()/train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['column', 'Percent'])
missing_data.index
missing_data.head(15)

,column,Percent
PropertyField29,200685,0.769636
PersonalField84,124208,0.476344
PropertyField38,1220,0.004679
PropertyField36,113,0.000433
PersonalField7,113,0.000433
PropertyField3,81,0.000311
PropertyField34,70,0.000268
PropertyField32,70,0.000268
PropertyField4,63,0.000242
PersonalField62,0,0.000000


In [9]:
total=testData.isnull().sum().sort_values(ascending = False)
percent = (testData.isnull().sum()/testData.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['column', 'Percent'])
missing_data.index
missing_data.head(15)

,column,Percent
PropertyField29,133945,0.770525
PersonalField84,82812,0.476380
PropertyField38,846,0.004867
PropertyField3,69,0.000397
PersonalField7,69,0.000397
PropertyField36,67,0.000385
PropertyField4,52,0.000299
PropertyField34,41,0.000236
PropertyField32,41,0.000236
PropertyField5,1,0.000006


In [10]:
numeric=['PropertyField29','PersonalField84']
train[numeric] = train[numeric].fillna(-1)
testData[numeric] = testData[numeric].fillna(-1)

In [11]:
Nac=['PropertyField32', 'PropertyField30', 'PropertyField4', 'PropertyField5', 
 'PropertyField34', 'PropertyField36', 'PropertyField38', 'PersonalField7', 'PropertyField3']

In [12]:
impage = SimpleImputer(missing_values=np.nan, strategy='most_frequent', fill_value=None) 
dfimpage = pd.DataFrame(impage.fit_transform(train[Nac]), columns=['imp_'+x for x in Nac],index=train.index)
train = pd.concat([train, dfimpage], axis=1)

In [13]:
train.drop(['PropertyField38','PersonalField7','PropertyField36', 'PropertyField3','PropertyField4','PropertyField34','PropertyField32',
    'PropertyField5','PropertyField30', 'PropertyField29','PersonalField84' ],axis=1,inplace=True)
train.head()

,QuoteNumber,Field6,Field7,Field8,Field9,Field10,Field11,Field12,CoverageField1A,CoverageField1B,...,Month,imp_PropertyField32,imp_PropertyField30,imp_PropertyField4,imp_PropertyField5,imp_PropertyField34,imp_PropertyField36,imp_PropertyField38,imp_PersonalField7,imp_PropertyField3
0,1,B,23,0.9403,0.0006,965,1.0200,N,17,23,...,8,Y,N,N,Y,Y,N,N,N,N
1,2,F,7,1.0006,0.0040,548,1.2433,N,6,8,...,4,N,N,N,Y,Y,N,N,N,N
2,4,F,7,1.0006,0.0040,548,1.2433,N,7,12,...,8,Y,N,Y,Y,Y,N,N,N,Y
3,6,J,10,0.9769,0.0004,1165,1.2665,N,3,2,...,4,Y,N,N,Y,N,N,N,N,N
4,8,E,23,0.9472,0.0006,1487,1.3045,N,8,13,...,1,N,N,Y,Y,N,N,N,N,Y


In [14]:
dfimp = pd.DataFrame(impage.transform(testData[Nac]), columns=['imp_'+x for x in Nac],index=testData.index)
testData = pd.concat([testData, dfimp], axis=1)
testData.drop(['PropertyField38','PersonalField7','PropertyField36', 'PropertyField3','PropertyField4','PropertyField34','PropertyField32',
    'PropertyField5','PropertyField30', 'PropertyField29','PersonalField84' ],axis=1,inplace=True)
testData.head()

,QuoteNumber,Field6,Field7,Field8,Field9,Field10,Field11,Field12,CoverageField1A,CoverageField1B,...,Month,imp_PropertyField32,imp_PropertyField30,imp_PropertyField4,imp_PropertyField5,imp_PropertyField34,imp_PropertyField36,imp_PropertyField38,imp_PersonalField7,imp_PropertyField3
0,3,E,16,0.9364,0.0006,1487,1.3045,N,4,4,...,8,Y,N,N,Y,N,N,N,N,N
1,5,F,11,0.9919,0.0038,564,1.1886,N,8,14,...,9,Y,N,N,Y,N,N,N,N,N
2,7,F,15,0.8945,0.0038,564,1.0670,N,11,18,...,3,Y,N,N,Y,Y,N,N,N,N
3,9,K,21,0.8870,0.0004,1113,1.2665,Y,14,22,...,3,N,N,N,Y,N,N,N,N,N
4,10,B,25,0.9153,0.0007,935,1.0200,N,4,5,...,12,Y,N,N,Y,N,Y,N,N,N


In [15]:
categoricalFeatures=["Field6","Field12","CoverageField8","CoverageField9","SalesField7","imp_PersonalField7","PersonalField16",
                    "PersonalField17","PersonalField18","PersonalField19","imp_PropertyField3","imp_PropertyField4","imp_PropertyField5",
                    "PropertyField7","PropertyField14","PropertyField28","imp_PropertyField30","PropertyField31","imp_PropertyField32",
                    "PropertyField33","imp_PropertyField34","imp_PropertyField36","PropertyField37","imp_PropertyField38","GeographicField63",
                     "GeographicField64"]

In [16]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(sparse=False, dtype=int,handle_unknown='ignore')
train1 = pd.DataFrame(ohe.fit_transform(train[categoricalFeatures]), columns=ohe.get_feature_names(),index=train.index)
train=pd.concat([train,train1],axis=1)
train.head()

,QuoteNumber,Field6,Field7,Field8,Field9,Field10,Field11,Field12,CoverageField1A,CoverageField1B,...,x22_Y,x23_N,x23_Y,x24_,x24_N,x24_Y,x25_CA,x25_IL,x25_NJ,x25_TX
0,1,B,23,0.9403,0.0006,965,1.0200,N,17,23,...,0,1,0,0,1,0,1,0,0,0
1,2,F,7,1.0006,0.0040,548,1.2433,N,6,8,...,0,1,0,0,1,0,0,0,1,0
2,4,F,7,1.0006,0.0040,548,1.2433,N,7,12,...,0,1,0,0,1,0,0,0,1,0
3,6,J,10,0.9769,0.0004,1165,1.2665,N,3,2,...,1,1,0,0,1,0,0,0,0,1
4,8,E,23,0.9472,0.0006,1487,1.3045,N,8,13,...,0,1,0,0,1,0,0,1,0,0


In [17]:
train.drop(["Field6","Field12","CoverageField8","CoverageField9","SalesField7","imp_PersonalField7","PersonalField16",
                    "PersonalField17","PersonalField18","PersonalField19","imp_PropertyField3","imp_PropertyField4","imp_PropertyField5",
                    "PropertyField7","PropertyField14","PropertyField28","imp_PropertyField30","PropertyField31","imp_PropertyField32",
                    "PropertyField33","imp_PropertyField34","imp_PropertyField36","PropertyField37","imp_PropertyField38","GeographicField63",
                     "GeographicField64"],axis=1,inplace=True)
train.head()

,QuoteNumber,Field7,Field8,Field9,Field10,Field11,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,...,x22_Y,x23_N,x23_Y,x24_,x24_N,x24_Y,x25_CA,x25_IL,x25_NJ,x25_TX
0,1,23,0.9403,0.0006,965,1.0200,17,23,17,23,...,0,1,0,0,1,0,1,0,0,0
1,2,7,1.0006,0.0040,548,1.2433,6,8,6,8,...,0,1,0,0,1,0,0,0,1,0
2,4,7,1.0006,0.0040,548,1.2433,7,12,7,12,...,0,1,0,0,1,0,0,0,1,0
3,6,10,0.9769,0.0004,1165,1.2665,3,2,3,2,...,1,1,0,0,1,0,0,0,0,1
4,8,23,0.9472,0.0006,1487,1.3045,8,13,8,13,...,0,1,0,0,1,0,0,1,0,0


In [18]:
dfim = pd.DataFrame(ohe.transform(testData[categoricalFeatures]), columns=ohe.get_feature_names(),index=testData.index)
testData = pd.concat([testData, dfim], axis=1)
testData.drop(["Field6","Field12","CoverageField8","CoverageField9","SalesField7","imp_PersonalField7","PersonalField16",
                    "PersonalField17","PersonalField18","PersonalField19","imp_PropertyField3","imp_PropertyField4","imp_PropertyField5",
                    "PropertyField7","PropertyField14","PropertyField28","imp_PropertyField30","PropertyField31","imp_PropertyField32",
                    "PropertyField33","imp_PropertyField34","imp_PropertyField36","PropertyField37","imp_PropertyField38","GeographicField63",
                     "GeographicField64"],axis=1,inplace=True)
testData.head()

,QuoteNumber,Field7,Field8,Field9,Field10,Field11,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,...,x22_Y,x23_N,x23_Y,x24_,x24_N,x24_Y,x25_CA,x25_IL,x25_NJ,x25_TX
0,3,16,0.9364,0.0006,1487,1.3045,4,4,4,4,...,1,1,0,0,0,1,0,1,0,0
1,5,11,0.9919,0.0038,564,1.1886,8,14,8,14,...,0,1,0,0,1,0,0,0,1,0
2,7,15,0.8945,0.0038,564,1.0670,11,18,11,18,...,0,1,0,0,1,0,0,0,1,0
3,9,21,0.8870,0.0004,1113,1.2665,14,22,15,22,...,0,1,0,0,1,0,0,0,0,1
4,10,25,0.9153,0.0007,935,1.0200,4,5,4,5,...,0,1,0,0,1,0,1,0,0,0


In [19]:
Xtrain = train
Xtest = testData
ytrain = trainData['QuoteConversion_Flag']

In [25]:
clf = DecisionTreeClassifier()
clf.fit(Xtrain, ytrain)
clf_predict=clf.predict(Xtest)
pred=pd.DataFrame({'QuoteNumber':testData.QuoteNumber,'QuoteConversion_Flag':clf_predict})
pred.to_csv('E:/Kaggle/insurance1.csv', index = None)

In [26]:
parameters={'criterion':['gini','entropy'],'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2),'min_samples_leaf':range(1,10,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(Xtrain, ytrain)
grid_parm=clf_random.best_params_
print(grid_parm)

{'min_samples_split': 90, 'min_samples_leaf': 5, 'max_depth': 13, 'criterion': 'entropy'}


In [27]:
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(Xtrain,ytrain)
clf_predict = clf.predict(Xtest)
pred=pd.DataFrame({'QuoteNumber':testData.QuoteNumber,'QuoteConversion_Flag':clf_predict})
pred.to_csv('E:/Kaggle/insurance2.csv', index = None)

In [28]:
rfc = RandomForestClassifier()
rfc.fit(Xtrain, ytrain)
rfc_predict=rfc.predict(Xtest)
pred=pd.DataFrame({'QuoteNumber':testData.QuoteNumber,'QuoteConversion_Flag':rfc_predict})
pred.to_csv('E:/Kaggle/insurancerf.csv', index = None)

In [29]:
parameters={ 'n_estimators': range(50,150,20),'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(Xtrain, ytrain)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)
rfc= RandomForestClassifier(**grid_parm_rfc)
rfc.fit(Xtrain,ytrain)
rfc_predict = rfc.predict(Xtest)
pred=pd.DataFrame({'QuoteNumber':testData.QuoteNumber,'QuoteConversion_Flag':rfc.predict(Xtest)})
pred.to_csv('E:/Kaggle/HYRF.csv', index = None)

{'n_estimators': 110, 'min_samples_split': 10, 'max_depth': 19}


In [30]:
#KNN
neigh = KNeighborsClassifier()
neigh.fit(Xtrain, ytrain)
neigh_predict=neigh.predict(Xtest)
pred=pd.DataFrame({'QuoteNumber':testData.QuoteNumber,'QuoteConversion_Flag':neigh.predict(Xtest)})
pred.to_csv('E:/Kaggle/KNNn.csv', index = None)

In [31]:
parameters={ 'leaf_size': range(5,60,5),'n_neighbors':range(1,10,1),'p':range(1,10,2)}
neigh_random = RandomizedSearchCV(neigh,parameters,n_iter=15)
neigh_random.fit(Xtrain, ytrain)
grid_parm_neigh=neigh_random.best_params_
print(grid_parm_neigh)

{'p': 1, 'n_neighbors': 1, 'leaf_size': 10}


In [32]:
neigh= KNeighborsClassifier(**grid_parm_neigh)
neigh.fit(Xtrain,ytrain)
neigh_predict = neigh.predict(Xtest)
pred=pd.DataFrame({'QuoteNumber':testData.QuoteNumber,'QuoteConversion_Flag':neigh.predict(Xtest)})
pred.to_csv('E:/Kaggle/HYPKNNnew.csv', index = None)

In [33]:
#multilayer per
mlf = MLPClassifier()
mlf.fit(Xtrain, ytrain)
mlf_predict=mlf.predict(Xtest)
pred=pd.DataFrame({'QuoteNumber':testData.QuoteNumber,'QuoteConversion_Flag':mlf.predict(Xtest)})
pred.to_csv('E:/Kaggle/mlpnew.csv', index = None)

In [ ]:
parameters={ 'max_iter': range(50,300,50),'n_iter_no_change':range(2,20,2)}
mlf_random = RandomizedSearchCV(mlf,parameters,n_iter=15)
mlf_random.fit(Xtrain, ytrain)
grid_parm_mlf=mlf_random.best_params_
print(grid_parm_mlf)
mlf= MLPClassifier(**grid_parm_mlf)
mlf.fit(Xtrain,ytrain)
mlf_predict = mlf.predict(Xtest)
pred=pd.DataFrame({'QuoteNumber':testData.QuoteNumber,'QuoteConversion_Flag':mlf.predict(Xtest)})
pred.to_csv('E:/Kaggle/Hmlfnew.csv', index = None)

{'n_iter_no_change': 12, 'max_iter': 250}


In [19]:
#SVC
slf=LinearSVC()
slf.fit(Xtrain, ytrain)
slf_pred = slf.predict(Xtest)
pred=pd.DataFrame({"QuoteNumber": testData.QuoteNumber ,"QuoteConversion_Flag": slf_pred})
pred.to_csv('E:/Kaggle/scvfnew.csv', index = None)

In [20]:
abc =GradientBoostingClassifier()
abc.fit(Xtrain, ytrain)
abc_predict=abc.predict(Xtest)
pred=pd.DataFrame({"QuoteNumber": testData.QuoteNumber ,"QuoteConversion_Flag": abc_predict})
pred.to_csv('E:/Kaggle/GBnew.csv', index = None)

In [21]:
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(ytrain))
sm = SMOTE(sampling_strategy='float', ratio=0.5)
Xres, yres = sm.fit_resample(Xtrain, ytrain)
print('Resampled dataset shape %s' % Counter(yres))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 211859, 1: 48894})
Resampled dataset shape Counter({0: 211859, 1: 105929})


In [23]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

models = [ GradientBoostingClassifier(), RandomForestClassifier(), DecisionTreeClassifier() ]
      
STrain, STest = stacking(models,                   
                           Xres, yres, Xtest,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

model =  DecisionTreeClassifier()
model = model.fit(STrain, yres)
y_pred = model.predict(STest)
pred=pd.DataFrame({'QuoteNumber':testData.QuoteNumber,'QuoteConversion_Flag':y_pred})
pred.to_csv('E:/Kaggle/stacking.csv', index = None)
#Get Prediction Probability for the predicted class as a dataframe
pred_Probability =pd.DataFrame(model.predict_proba(STest))

pred_Probability.head()


___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [GradientBoostingClassifier]
    fold  0:  [0.93629796]
    fold  1:  [0.93530278]
    fold  2:  [0.93496293]
    fold  3:  [0.93483624]
    ----
    MEAN:     [0.93534998] + [0.00057328]
    FULL:     [0.93534998]

model  1:     [RandomForestClassifier]
    fold  0:  [0.91871413]
    fold  1:  [0.91635933]
    fold  2:  [0.91550342]
    fold  3:  [0.92086449]
    ----
    MEAN:     [0.91786034] + [0.00209535]
    FULL:     [0.91786033]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.90660558]
    fold  1:  [0.90868126]
    fold  2:  [0.90548416]
    fold  3:  [0.90723259]
    ----
    MEAN:     [0.90700090] + [0.00115478]
    FULL:     [0.90700089]



,0,1
0,0.951459,0.048541
1,0.951459,0.048541
2,0.951459,0.048541
3,0.951459,0.048541
4,0.951459,0.048541
